In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## **Data Wrangling**

Data Gathering

In [ ]:
!git clone https://github.com/daytapy/Customer_Churn.git /content/Customer_Churn

In [ ]:
file_path = '/content/Customer_Churn/Telco_customer_churn_adapted_v2.xlsx'
telco_df = pd.read_excel(file_path)
telco_df.head()

Assesing Data

In [ ]:
telco_df.info()

In [ ]:
telco_df.describe()

In [ ]:
print('Jumlah Data Duplikat : ', telco_df.duplicated().sum())

In [ ]:
korelasi = telco_df.corr()
sns.heatmap(korelasi, annot=True, cmap='YlGnBu')
plt.show()

# **Exploratory Data Analysis**

In [ ]:
#Menambahkan underscore pada setiap nama kolom agar dapat di panggil
telco_df = telco_df.rename(columns=lambda x: x.replace(' ','_'))
telco_df.head()

In [ ]:
telco_df = telco_df.rename(columns={'Monthly_Purchase_(Thou._IDR)':'Monthly_Purchase', 'CLTV_(Predicted_Thou._IDR)':'CLTV'})
telco_df.head()

In [ ]:
telco_df['Total_Charges'] = telco_df['Monthly_Purchase'] * telco_df['Tenure_Months']
telco_df.head()

In [ ]:
def categorize_tenure(tenure):
    if 1 <= tenure <= 11:
        return '< 1 Tahun'
    elif tenure == 12:
        return '1 Tahun'
    elif 12 < tenure <= 24:
        return '2 Tahun'
    elif 24 < tenure <= 36:
        return '3 Tahun'
    else:
        return '> 3 Tahun'
    # Tambahkan kategori lain jika diperlukan

# Buat kolom baru 'contract_type' berdasarkan 'tenure_month'
telco_df['Contract_Type'] = telco_df['Tenure_Months'].apply(categorize_tenure)
telco_df.head()

***Bagaimana perbandingan jumlah customer yg melakukan churn dengan yang tidak melakukan churn?***

In [ ]:
churn_plot = telco_df.groupby(by='Churn_Label').Customer_ID.nunique().sort_values(ascending=False).reset_index()
churn_plot

In [ ]:
fig = px.pie(
    churn_plot,
    values='Customer_ID',
    names='Churn_Label',
    color='Churn_Label',
    color_discrete_map={'Yes':'cyan',
                        'No':'darkblue'}
)
fig.show()

Dilihat dari pie chart diatas, bahwa jumlah customer yang melakukan churn (berpindah ke operator lain) sebesar 26,5%. Sedangkan jumlah customer yang tidak melakukan churn (masih tetap di operator ini) sebesar 73,5%.

***Bagaimana lokasi geografis pelanggan yang kita miliki?***

In [ ]:
fig = px.scatter_mapbox(telco_df.groupby(['Latitude','Longitude'])['Customer_ID'].count().reset_index(), lat="Latitude", lon="Longitude", hover_data= ['Customer_ID'], zoom=4, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
telco_df.groupby(by='Location').Customer_ID.nunique().sort_values(ascending=False).reset_index()


*   Customer yang kita miliki berasal dari 2 lokasi yaitu Jakarta dan Bandung
*   Customer terbanyak berasal dari Jakarta



***Bagaimana produk tambahan seperti Games Product, Music Product, Education Product, Call Center, Video Product, dan Use MyApp memengaruhi tingkat churn?***

In [ ]:
fig, axes = plt.subplots(2,3, figsize = (15,7), sharey = True)
plt.suptitle("Pengaruh Produk Tambahan Terhadap Tingkat Churn")
sns.countplot(x = "Games_Product", data = telco_df, hue = "Churn_Label", ax=axes[0,0], palette="Set1")
sns.countplot(x = "Music_Product", data = telco_df, hue = "Churn_Label", ax=axes[0,1], palette="Set1")
sns.countplot(x = "Education_Product", data = telco_df, hue = "Churn_Label", ax=axes[0,2], palette="Set1")
sns.countplot(x = "Call_Center", data = telco_df, hue = "Churn_Label", ax=axes[1,0], palette="Set1")
sns.countplot(x = "Video_Product", data = telco_df, hue = "Churn_Label", ax=axes[1,1], palette="Set1")
sns.countplot(x = "Use_MyApp", data = telco_df, hue = "Churn_Label", ax=axes[1,2], palette="Set1")
plt.show()



*   Penggunaan Games Product, Music Product, Education Product dan Call Center
tampaknya memiliki sedikit pengaruh terhadap tingkat churn, dengan tingkat churn yang relatif rendah artinya operator ini cukup baik dalam layanan produk2 tambahan tersebut.
*   Dan juga sama dengan penggunaan Video Product dan Use MyApp, memiliki tingkat churn yang relatif rendah meskipun lebih besar sedikit dari 4 produk tambahan lainnya.



***Bagaimana tingkat churn berkaitan dengan lamanya customer berlangganan (Tenure Months)?***

In [ ]:
plt.figure(figsize=(8, 4))
sns.kdeplot(data=telco_df, x="Tenure_Months", hue="Churn_Label", common_norm=False, fill=True, palette="Set1")
plt.title("Distribusi Tingkat Churn berdasarkan Lamanya Berlangganan (Tenure Months)")
plt.xlabel("Tenure Months")
plt.ylabel("Density")
plt.legend(title="Churn", labels=["Tidak Churn", "Churn"])
plt.show()



*   Tingkat churn cenderung lebih tinggi pada awal langganan (Tenure Months rendah) dan kemudian mengalami penurunan seiring dengan berjalannya waktu. Hal ini menunjukkan bahwa customer yang baru bergabung cenderung memiliki tingkat churn yang lebih tinggi.
*   Sedangkan customer yang telah berlangganan lebih lama cenderung lebih stabil dalam menggunakan layanan perusahaan telekomunikasi.

Oleh karena itu, dapat disimpulkan bahwa lamanya customer berlangganan memiliki pengaruh signifikan terhadap tingkat churn dalam perusahaan tersebut.



In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(data=telco_df, x='Contract_Type', hue='Churn_Label', palette="Set2")
plt.title("Hubungan Antara Lama Berlangganan Dengan Tingkat Churn")
plt.xticks(fontsize=11)
plt.xlabel("Lama Berlangganan", fontsize=11)
plt.show()

***Bagaimana tingkat churn berkaitan dengan pembelian bulanan customer (Monthly Purchase)?***

In [ ]:
plt.figure(figsize=(8, 4))
sns.kdeplot(data=telco_df, x="Monthly_Purchase", hue="Churn_Label", common_norm=False, fill=True, palette="cubehelix")
plt.title("Distribusi Tingkat Churn berdasarkan pembelian bulanan (Monthly Purchase)")
plt.xlabel("Monthly Purchase")
plt.ylabel("Density")
plt.legend(title="Churn", labels=["Tidak Churn", "Churn"])

Customer yang melakukan transaksi per bulan rata-rata di atas Rp.75000, akan lebih cenderung melakukan churn dibandingkan dengan rata-rata transaksi di bawahnya.

***Apakah ada hubungan antara jenis pembayaran (Payment Method) dan tingkat churn?***

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(data=telco_df, x='Payment_Method', hue='Churn_Label', palette='muted')
plt.title("Hubungan Antara Jenis Pembayaran Dengan Tingkat Churn")
plt.xticks(fontsize=11)
plt.xlabel("Jenis Pembayaran", fontsize=11)
plt.show()

Customer yang menggunakan metode pembayaran Pulsa lebih cenderung mengalami churn dibandingkan dengan customer yang menggunakan metode pembayaran lainnya

***Apakah ada hubungan antara jenis perangkat dengan tingkat churn?***

In [ ]:
plt.figure(figsize=(8, 4))
sns.countplot(data=telco_df, x='Device_Class', hue='Churn_Label', palette="pastel")
plt.title("Hubungan Antara Jenis Perangkat Dengan Tingkat Churn")
plt.xticks(fontsize=11)
plt.xlabel("Jenis Perangkat", fontsize=11)
plt.show()

Customer yang menggunakan perangkat dengan spesifikasi tinggi cenderung mengalami churn dibandingkan dengan customer yang menggunakan perangkat dengan spesifikasi menengah dan rendah.

# **Data Preparation**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [ ]:
telco_df.columns

In [ ]:
#Menghapus kolom yang tidak digunakan
columns_to_drop = ['Customer_ID', 'Longitude', 'Latitude','CLTV']
df = telco_df.drop(columns=columns_to_drop, axis=1)
df = pd.get_dummies(df)
columns_to_drop = ['Location_Bandung', 'Games_Product_No', 'Music_Product_No', 'Education_Product_No', 'Call_Center_No', 'Video_Product_No', 'Use_MyApp_No', 'Churn_Label_No']
df = df.drop(columns=columns_to_drop, axis=1)

In [ ]:
df_train = df.sample(frac=0.8, random_state=42)
df_tmp = df.drop(df_train.index)

df_test = df_tmp.sample(frac=0.5, random_state=42)
df_val = df_tmp.drop(df_test.index)

In [ ]:
x_train = df_train
y_train = df_train.pop('Churn_Label_Yes')
x_test = df_test
y_test = df_test.pop('Churn_Label_Yes')
x_val = df_val
y_val = df_val.pop('Churn_Label_Yes')

In [ ]:
labels_train = df_train.columns
labels_test = df_test.columns
labels_val = df_val.columns
normalisasi = sklearn.preprocessing.StandardScaler()
train_normalisasi = normalisasi.fit(x_train).transform(x_train)
test_normalisasi = normalisasi.fit(x_test).transform(x_test)
val_normalisasi = normalisasi.fit(x_val).transform(x_val)
X_train = pd.DataFrame(train_normalisasi, columns=labels_train)
X_test = pd.DataFrame(test_normalisasi, columns=labels_test)
X_val = pd.DataFrame(val_normalisasi, columns=labels_val)
X_train.head()

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(17, input_shape=(27,), activation='relu'),  # Gantilah input_shape sesuai dengan data Anda
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.BatchNormalization(),

    # tf.keras.layers.Dense(4, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    # tf.keras.layers.Dropout(0.1),

    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

In [ ]:
smote = SMOTE(sampling_strategy=0.46, k_neighbors=10, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
history1 = model.fit(
    X_train_res,
    y_train_res,
    epochs=100,
    # validation_split= 0.2,
    shuffle=True,
    validation_data=(X_test, y_test),
    # callbacks=callbacks_list,
    batch_size = 256,
    verbose=1
)

In [ ]:
hist = pd.DataFrame(history1.history)
hist['epoch'] = history1.epoch
hist.tail()
def plot_loss(history):
  plt.plot(history1.history['loss'], label='loss')
  plt.plot(history1.history['val_loss'], label='val_loss')
  plt.ylim([0.1, 1])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Churn]')
  plt.legend()
  plt.grid(True)

plot_loss(history1)

In [ ]:
test_loss= model.evaluate(X_val, y_val, verbose=1)
print('\nTest', test_loss)

In [ ]:
# predict the churn values
y_pred = model.predict(X_val)
# unscaling the ypred values
y_pred_lis = []
for i in y_pred:
    if i>0.5:
        y_pred_lis.append(1)
    else:
        y_pred_lis.append(0)

In [ ]:
# checking for performance metrices
#importing classification_report and confusion metrics
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sb
#print classification_report
print(classification_report(y_val,y_pred_lis))
# ploting the confusion metrix plot
conf_mat = tf.math.confusion_matrix(labels=y_val,predictions=y_pred_lis)
plt.figure(figsize = (17,7))
sb.heatmap(conf_mat, annot=True,fmt='d')
plt.xlabel('Predicted_number')
plt.ylabel('True_number')